In [22]:
import json
import networkx as nx
import numpy as np

import functions

In [23]:
#function to read .json file of the input graph
def readPreferenceGraph(filePath) :
    with open(filePath) as file :
        data = json.load(file)

    G = nx.MultiDiGraph()

    for edge in data["edges"] :
        G.add_edge(edge["source"], edge["target"], opinion=edge["opinion"], questionItem=edge["question item"])
    
    return G

In [24]:
def createInteractionGraph(preferenceGraph, threshold, maxInteractions, maxTimestamp) :
    n = preferenceGraph.number_of_nodes()
    activitiesCount = 6

    interactionGraph = []
    interactionGraphIntensity = []
    
    degrees = [0] * n
   
        
    # list of possible peers to inteact for each person
    possiblePeers = functions.analyzeQuiz(prefGraph=preferenceGraph, \
                                          threshold=threshold, \
                                          numNodes=n)

    # ------------------------------------- #
    #    interactions at timetsamp t = 0    #
    # ------------------------------------- #
    # 1st step: interactions based on a scale-free network model with m=3
    interactionGraph.append(nx.barabasi_albert_graph(n=preferenceGraph.number_of_nodes(), \
                                                     m=3))

    # 2nd step: add random activities as weights
    for u, v in interactionGraph[0].edges() :
        interactionGraph[0][u][v]["interaction_index"] = np.random.choice(range(activitiesCount))
    
    # 3rd step: calculate intensity
    snapshotIntensity = functions.calculateIntensityForSnapshot(snapshotGraph=interactionGraph[0], 
                                                                numNodes=n)
    interactionGraphIntensity.append(snapshotIntensity)

    # 4th step: add snapshot degrees to the total number of degrees
    for node in range(n) :
        degrees[node] += interactionGraph[0].degree(node) 
    

    # -------------------------------------- #
    #    interactions at timetsamp t >= 1    #
    # -------------------------------------+ #
    for time in range(1, maxTimestamp + 1) :
        
        # 1st step: create new snapshot
        # 2nd step: add activities as weights
        snapshot = functions.createSnapshot(numNodes=n, \
                                            maxInteractions=maxInteractions, \
                                            degrees=degrees, \
                                            possiblePeers=possiblePeers, \
                                            activitiesCount=activitiesCount)
        interactionGraph.append(snapshot)
        
        # 3rd step: calculate intensity
        snapshotIntensity = functions.calculateIntensityForSnapshot(snapshotGraph=interactionGraph[time], \
                                                                    numNodes=n)
        interactionGraphIntensity.append(snapshotIntensity)
        
        # 4th step: add snapshot degrees to the total number of degrees
        for node in range(n) :
            degrees[node] += interactionGraph[time].degree(node) 

    
    # calculate indexes for each snapshot
    graphSequenceIndexes = []
    for time in range(maxTimestamp + 1) :
        snapshotIndexes = functions.calculateIndexes(prefGraph=preferenceGraph, \
                                                     snapshotGraph=interactionGraph[time], \
                                                     numNodes=n)
        graphSequenceIndexes.append(snapshotIndexes)


    return {
        "graph" : interactionGraph,
        "sequence intensity": interactionGraphIntensity,
        "individual": {
            "popularity": np.average([item["individual"]["popularity"] for item in graphSequenceIndexes]),
            "antipathy": np.average([item["individual"]["antipathy"] for item in graphSequenceIndexes]),
            "affective connection": np.average([item["individual"]["affective connection"] for item in graphSequenceIndexes]),
            "sociometric status": np.average([item["individual"]["sociometric status"] for item in graphSequenceIndexes]),
            "positive expansion": np.average([item["individual"]["positive expansion"] for item in graphSequenceIndexes]),
            "negative expansion": np.average([item["individual"]["negative expansion"] for item in graphSequenceIndexes]),
            "realistic perception": np.average([item["individual"]["realistic perception"] for item in graphSequenceIndexes])
        },
        "group": {
            "association": np.average([item["group"]["association"] for item in graphSequenceIndexes]),
            "dissociation": np.average([item["group"]["dissociation"] for item in graphSequenceIndexes]),
            "cohesion": np.average([item["group"]["cohesion"] for item in graphSequenceIndexes]),
            "social intensity": np.average([item["group"]["social intensity"] for item in graphSequenceIndexes]),
        }
    }

In [25]:
path = "./preferenceGraph.json"

options = {
    "threshold" : 0.4,
    "maxInteractions" : 200,
    "maxTimestamp" : 50
}

G = createInteractionGraph(preferenceGraph=readPreferenceGraph(path), **options)

print("\033[1mSnapshots added...\033[0m")
for i in range(6) :
    print(f"at t = {i}: {G['graph'][i]}")

print("\n\033[1mGraph Sequence Intensity\033[0m")
print(G['sequence intensity'])

print("\n\033[1mIndividual Indexes\033[0m")
print(G["individual"])

print("\n\033[1mGroup Indexes\033[0m")
print(G["group"])

Snapshots added...
at t = 0: Graph with 100 nodes and 291 edges
at t = 1: Graph with 100 nodes and 195 edges
at t = 2: Graph with 100 nodes and 292 edges
at t = 3: Graph with 100 nodes and 199 edges
at t = 4: Graph with 100 nodes and 197 edges
at t = 5: Graph with 100 nodes and 300 edges

Graph Sequence Intensity
[13487, 4310, 9086, 4428, 4504, 9676, 4441, 4343, 4433, 4495, 4171, 4326, 9269, 9290, 9252, 9447, 4191, 9044, 4210, 9277, 4254, 9389, 9576, 4390, 9371, 4369, 9557, 9501, 9457, 9227, 4241, 4589, 4584, 9373, 9304, 9406, 8623, 6902, 2321, 2221, 1973, 2784, 891, 1471, 844, 354, 442, 160, 200, 90, 68]

Individual Indexes
{'popularity': 0.02703109526638938, 'antipathy': 0.013824519706872648, 'affective connection': 0.23208331809802396, 'sociometric status': 0.0180154486036839, 'positive expansion': 0.027031095266389388, 'negative expansion': 0.013824519706872648, 'realistic perception': 0.21789352362014847}

Group Indexes
{'association': 0.007494553376906319, 'dissociation': 0.00122

In [28]:
timestamp = 1
print(f"\033[1mInteraction activities at timestamp = {timestamp}\033[0m")
for i, j, data in G["graph"][timestamp].edges(data=True) :
    print(f"({i},{j}) => interaction: {functions.getActivity(data['interaction_index'])}")

Interaction activities at timestamp = 1
(0,86) => interaction: to share content, chat in a social network platform
(0,77) => interaction: to be volunteers
(0,49) => interaction: to be teammates in a sports team
(0,71) => interaction: to share content, chat in a social network platform
(1,59) => interaction: to play games
(1,29) => interaction: to be teammates in a sports team
(1,19) => interaction: to be volunteers
(1,30) => interaction: to play games
(1,50) => interaction: to be teammates in a sports team
(1,54) => interaction: to share content, chat in a social network platform
(2,69) => interaction: to participate in debate/group presentation & discussion
(2,30) => interaction: to be teammates in a sports team
(3,44) => interaction: to participate in debate/group presentation & discussion
(3,72) => interaction: to study in group or work on collaborative projects
(3,4) => interaction: to be volunteers
(3,48) => interaction: to play games
(4,21) => interaction: to play games
(4,23) =>